In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import savgol_filter
from datetime import datetime

In [ ]:
MEL_14DAY_AVG_VALUE = '34'


PROJECT_ID = '<<PROJECT_ID>>'

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID)


## New case details


In [ ]:
df_eu = client.query(f""" 
WITH 
regions_over_100 AS (
-- Filter to only regions that had > 100 new daily cases
SELECT DISTINCT country_name, subregion1_name
FROM 
`bigquery-public-data.covid19_open_data.covid19_open_data` 
WHERE new_confirmed > 100),

new_case_counts AS (
-- Select approx. 596 provinces worldwide
SELECT 
A.date,
A.country_name,
A.subregion1_name || ', ' || A.country_name as province_name,
SUM(A.new_confirmed) as new_cases
FROM 
`bigquery-public-data.covid19_open_data.covid19_open_data` A
INNER JOIN regions_over_100 B
  ON A.country_name = B.country_name
  AND A.subregion1_name = B.subregion1_name
WHERE
  A.country_name <> 'Australia'
  AND A.subregion1_name IS NOT NULL
  AND A.subregion2_name IS NULL
  AND A.date < CAST('2020-08-01' as DATE)
GROUP BY 1, 2, 3

UNION ALL 

-- Australia (include Melbourne 2nd wave)
SELECT 
date,
country_name,
subregion1_name || ', ' || country_name as province_name,
new_confirmed as new_cases
FROM 
`bigquery-public-data.covid19_open_data.covid19_open_data` 
WHERE
  country_name = 'Australia'
  
-- New Zealand (Auckland)
UNION ALL 

SELECT 
date,
country_name,
country_name as province_name,
SUM(new_confirmed) as new_cases
FROM 
`bigquery-public-data.covid19_open_data.covid19_open_data` 
WHERE
  country_name = 'New Zealand'
GROUP BY 1, 2, 3),

-- Apply moving average smoothing (14-day average)

new_cases_smooth AS (
SELECT
*,
AVG(new_cases) OVER (PARTITION BY province_name ORDER BY date ROWS BETWEEN 13 PRECEDING AND CURRENT ROW) as new_cases_moving_avg
FROM new_case_counts
WHERE new_cases IS NOT NULL),

-- Find indication whether over intial peak
max_cases_smooth_date AS (SELECT * FROM (
SELECT 
province_name,
date,
new_cases_moving_avg,
ROW_NUMBER() OVER (PARTITION BY province_name ORDER BY new_cases_moving_avg DESC) as rn
FROM new_cases_smooth
) t WHERE t.rn = 1),

new_cases_smooth_peak_ind AS (
SELECT 
A.*,
B.date as peak_new_cases_date,
B.new_cases_moving_avg as peak_new_cases_moving_avg,
CASE WHEN A.date > B.date THEN 1 ELSE 0 END as after_peak_flag
FROM new_cases_smooth A 
LEFT JOIN max_cases_smooth_date B 
  ON A.province_name = B.province_name),

-- Find days with similar 14-day average and on a post-peak trend
equiv_14day_average AS (SELECT * FROM (
SELECT 
province_name,
date,
ROW_NUMBER() OVER (PARTITION BY province_name ORDER BY ABS(new_cases_moving_avg - 36)) as rn
FROM new_cases_smooth_peak_ind
WHERE after_peak_flag = 1
) t WHERE t.rn = 1),

new_cases_14day_avg_shifted AS (
SELECT 
A.*,
B.date as equiv_14day_average_date,
DATE_DIFF(A.date, B.date, DAY) as days_from_equiv_14day_avg_val
FROM new_cases_smooth_peak_ind A 
LEFT JOIN equiv_14day_average B 
  ON A.province_name = B.province_name)
  
 SELECT 
date,
country_name,
province_name,
new_cases,
new_cases_moving_avg,
peak_new_cases_date,
peak_new_cases_moving_avg,
after_peak_flag,
equiv_14day_average_date,
days_from_equiv_14day_avg_val
FROM new_cases_14day_avg_shifted
ORDER BY province_name, date






""").to_dataframe()
df_eu['date'] = pd.to_datetime(df_eu['date'])
df_eu['peak_new_cases_date'] = pd.to_datetime(df_eu['peak_new_cases_date'])

In [ ]:
df_eu.to_csv('df_world_new_cases.csv', index=False)